In [1]:
!nvidia-smi

Sat Dec 14 16:49:38 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

**1. Install Req libs**

In [2]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 6.4 MB/s eta 0:00:00


In [3]:
!pip install -q transformers einops accelerate langchain bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 8.5 MB/s eta 0:00:00


In [4]:
!pip install sentence_transformers

In [5]:
!pip install llama-index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.8/195.8 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.3/390.3 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5


In [9]:
!pip install llama-index-llms-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.8/402.8 kB 10.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.26.5
    Uninstalling huggingface-hub-0.26.5:
      Successfully uninstalled huggingface-hub-0.26.5


In [23]:
pip install docx2txt

  Preparing metadata (setup.py) ... done
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3960 sha256=230b9aad46e4464933412caf7e2cbac024f04da0bbe6125132ebad14a7e42ae5
  Stored in directory: /root/.cache/pip/wheels/22/58/cf/093d0a6c3ecfdfc5f6ddd5524043b88e59a9a199cb02352966
Successfully built docx2txt


**2. Load packages**

VectorStoreIndex - used to create a local db

SimpleDirectoryReader - used to create the data extraction part and doc loading 


In [17]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import PromptTemplate

**3. Data handling**

In [18]:
!mkdir Data

In [24]:
documents = SimpleDirectoryReader("/content/Data").load_data()

In [27]:
documents[0].text

'Introduction to Generative AI\n\nGenerative AI (Gen AI)\n\nGenerative AI is a type of artificial intelligence that can create new content, like writing, images, music, or even code, instead of just recognizing patterns or making predictions. It learns from a lot of data and uses that knowledge to produce something new and original, which can look or sound like the data it was trained on. \n\n\n\nHow Gen AI is differing from ML, DL, NLP and LLM?\n\n\n\n\n\nLLM (Large Language Model)\n\nA Large Language Model (LLM) is a type of artificial intelligence model designed to understand and generate human language. It is built using a neural network that is trained on massive amounts of text data, allowing it to perform a wide variety of language-related tasks, such as answering questions, writing essays, translating languages, summarizing information, and even generating creative content like stories or poems.\n\nComparison of LLM and Gen AI:\n\nAspect\n\n\n\nGenerative AI (Gen AI)\n\n\n\nLar

In [28]:
len(documents)

2

**4. Prompt Creation**

In [31]:
system_prompt = """<|SYSTEM|>
You are an Q&A assistant. your goal is to answer the user based on the question.
Give as much details as possible and accurate answers as well.
If you don't know the answer, just say that you don't know.
Perform based on the context and question.
"\n\n"
"{context}"
"""

query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

**5. Hugging Face crediential check**

In [32]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in c

**6. HF Model initialization**

In [34]:
import torch

llm = HuggingFaceLLM(
    context_window=4096, # input size of llm 
    max_new_tokens=256, 
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="mistralai/Mistral-7B-Instruct-v0.1",
    model_name="mistralai/Mistral-7B-Instruct-v0.1",
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

**7. Create Embedding model**

In [35]:
!pip install llama-index-embeddings-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.6/915.6 kB 18.6 MB/s eta 0:00:00


In [36]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Initialize the embedding model
embedding_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

###  to create a vector store

In [42]:
from llama_index.core import Settings

Settings.chunk_size = 1024
Settings.chunk_overlap = 20
Settings.llm = llm
Settings.embed_model = embedding_model

# Use the settings object wherever needed in your code

**8. Create an Index an store it in local storage**

In [43]:
index = VectorStoreIndex.from_documents(documents)

In [44]:
index.storage_context.persist()

**9. Create an query engine**

In [45]:
query_engine = index.as_query_engine()

In [46]:
response = query_engine.query("What is llamaindex?")
response

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Response(response='LLAMA (Large Language Model Architecture) is a type of artificial intelligence model designed to understand and generate human language. It is built using a neural network that is trained on massive amounts of text data, allowing it to perform a wide variety of language-related tasks, such as answering questions, writing essays, translating languages, summarizing information, and even generating creative content like stories or poems. LLAMA is a subset of Generative AI, specifically dealing with text.', source_nodes=[NodeWithScore(node=TextNode(id_='fa0f7ec0-6be9-4fbb-9dfe-456a387971c4', embedding=None, metadata={'file_name': '1. Introduction to Generative AI.docx', 'file_path': '/content/Data/1. Introduction to Generative AI.docx', 'file_type': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document', 'file_size': 431129, 'creation_date': '2024-12-14', 'last_modified_date': '2024-12-14'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file

**10. To get an answer in an understandable way**

In [47]:
from IPython.display import Markdown, display
display(Markdown(f"<b>{response}</b>"))

<b>LLAMA (Large Language Model Architecture) is a type of artificial intelligence model designed to understand and generate human language. It is built using a neural network that is trained on massive amounts of text data, allowing it to perform a wide variety of language-related tasks, such as answering questions, writing essays, translating languages, summarizing information, and even generating creative content like stories or poems. LLAMA is a subset of Generative AI, specifically dealing with text.</b>

In [48]:
response = query_engine.query("What is Genai?")
response

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Response(response='Gen AI (Generative AI) is a type of artificial intelligence that can create new content, like writing, images, music, or even code, instead of just recognizing patterns or making predictions. It learns from a lot of data and uses that knowledge to produce something new and original, which can look or sound like the data it was trained on.', source_nodes=[NodeWithScore(node=TextNode(id_='fa0f7ec0-6be9-4fbb-9dfe-456a387971c4', embedding=None, metadata={'file_name': '1. Introduction to Generative AI.docx', 'file_path': '/content/Data/1. Introduction to Generative AI.docx', 'file_type': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document', 'file_size': 431129, 'creation_date': '2024-12-14', 'last_modified_date': '2024-12-14'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modifi

In [49]:
from IPython.display import Markdown, display
display(Markdown(f"<b>{response}</b>"))

<b>Gen AI (Generative AI) is a type of artificial intelligence that can create new content, like writing, images, music, or even code, instead of just recognizing patterns or making predictions. It learns from a lot of data and uses that knowledge to produce something new and original, which can look or sound like the data it was trained on.</b>

In [50]:
response = query_engine.query("What is transfomer?")
response

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Response(response='Transformer is an advanced model for sequence-to-sequence processing that excels at handling sequential tasks. It processes input data one word at a time, maintaining context over time. Unlike LSTMs, Transformers process sequences of words simultaneously, offering higher efficiency. They handle data in parallel, making them faster and more suitable for complex tasks.', source_nodes=[NodeWithScore(node=TextNode(id_='6a1da898-3b7c-4623-aaa7-149202225816', embedding=None, metadata={'file_name': 'Transformer.docx', 'file_path': '/content/Data/Transformer.docx', 'file_type': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document', 'file_size': 1963835, 'creation_date': '2024-12-14', 'last_modified_date': '2024-12-14'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'l

In [51]:
from IPython.display import Markdown, display
display(Markdown(f"<b>{response}</b>"))

<b>Transformer is an advanced model for sequence-to-sequence processing that excels at handling sequential tasks. It processes input data one word at a time, maintaining context over time. Unlike LSTMs, Transformers process sequences of words simultaneously, offering higher efficiency. They handle data in parallel, making them faster and more suitable for complex tasks.</b>

**11. Final Output**

In [54]:
from IPython.display import Markdown, display

while True:
  user_query = input("Enter your query: ")
  if user_query.lower() == "q" or user_query.lower() == "e":
    print("Query Not Entered So Exiting!.. Good Bye")
    break

  response = query_engine.query(user_query)
  display(Markdown(f"<b>{response}</b>"))

Enter your query: who is malavika


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<b>Malavika is a name of a person. It is a female given name of Indian origin. It is derived from the Sanskrit word "mala" meaning "garland" or "necklace". It is also a common name in many countries, especially in India, Nepal, and Bangladesh.</b>

Enter your query: what is chat gpt


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<b>ChatGPT is a large language model developed by OpenAI. It is designed to understand and generate human language, and can be used for a wide range of tasks such as answering questions, writing essays, and even generating creative content like stories or poems.</b>

Enter your query: transformer archotectire


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<b>The Transformer architecture is a type of neural network designed for sequence-to-sequence processing tasks. It was introduced in a paper by Vaswani et al. in 2017 and has since become a popular choice for natural language processing tasks. The Transformer architecture is based on the idea of self-attention, which allows the model to focus on different parts of the input sequence when making predictions. This makes it more efficient and effective than traditional recurrent neural network architectures like LSTMs.</b>

Enter your query: e
Query Not Entered So Exiting!.. Good Bye
